In [ ]:
!pip install nltk spacy

In [ ]:
!python -m spacy download en_core_web_sm
# !python -m spacy link xx_ent_wiki_sm xx

In [ ]:
!pip install langid

In [ ]:
import nltk
import spacy
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
nlp = spacy.load('xx', disable=['ner', 'parser'])

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [ ]:
article_info.title = article_info.title.str.lower()
article_info.paperAbstract = article_info.paperAbstract.str.lower()

In [ ]:
import langid # language identification (i.e. what language is this?)
from nltk.classify.textcat import TextCat # language identification from NLTK

In [ ]:
# taken from https://www.kaggle.com/rtatman/analyzing-multilingual-data
# lang_ids = article_info['title'].apply(langid.classify)
langs = lang_ids.apply(lambda tuple: tuple[0])

# how many unique language labels were applied?
print("Number of tagged languages (estimated):")
print(len(langs.unique()))

# percent of the total dataset in English
print("Percent of data in English (estimated):")
print((sum(langs=="en")/len(langs))*100)

In [ ]:
# Number of tagged languages (estimated):
# 62
# Percent of data in English (estimated):
# 96.87297408520259

In [ ]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in article_info['title'] if article_info['abstract'] is not np.nan)

In [ ]:
article_info.columns

In [ ]:
article_info.fieldsOfStudy = article_info.fieldsOfStudy.apply(lambda x: np.nan if x[0] is np.nan else x)

In [ ]:
mlb = MultiLabelBinarizer()

X = mlb.fit_transform(article_info.fieldsOfStudy)

article_info = article_info.join(pd.DataFrame(X, columns=mlb.classes_, index=article_info.index))

In [ ]:
article_info.drop('fieldsOfStudy', axis=1, inplace=True)

In [ ]:
mlb = MultiLabelBinarizer()

X = mlb.fit_transform(article_info.sources)

article_info = article_info.join(pd.DataFrame(X, columns=mlb.classes_, index=article_info.index))

In [ ]:
article_info.drop('sources', axis=1, inplace=True)

In [ ]:
columns_to_remove = ['outCitations', 's2Url', 'pdfUrls', 'journalVolume', 'venue', 'journalName', 'doi', 'doiUrl', 's2PdfUrl', 'pmid', 'magId', 'entities']

In [ ]:
article_info.drop(columns_to_remove, axis=1, inplace=True)

In [ ]:
article_info.journalName.isna().sum()

In [ ]:
article_info.venue.value_counts()[article_info.venue.value_counts() > 20]

In [ ]:
article_info.journalName.value_counts()

In [ ]:
# features for one-hot encoding
# venue
# journalName


from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(article_info.journalName.value_counts().index.values)
article_info.journalName = enc.transform(article_info.journalName.values)

In [ ]:
enc = LabelEncoder()
enc.fit(article_info.venue.value_counts().index.values)
article_info.venue = enc.transform(article_info.venue.values)

In [ ]:
article_info.shape

In [ ]:
article_info.head(3)

In [ ]:
article_info.title.apply(lambda row: len(row))

In [ ]:
%%time

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_process=2)]

# print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
data_clean = pd.DataFrame({'clean': txt})
data_clean = data_clean.dropna().drop_duplicates()
data_clean.shape